## Postalcode, Borough, Neighborhoods, Latitude and Longitude in Toronto

This notebook obtains data of Toronto (Postalcode, Borough and Neighborhoods) from an URL and transforms it into a pandas dataframe for further analysis. Then it adds the Latitude and Longitude to it.

In [1]:
#Import necessary libraries
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text

In [3]:
#Extract text in xml language 
soup = BeautifulSoup(source, 'xml')

In [4]:
#Extract the table content
table=soup.find('table')

In [5]:
#Create dataframe with three columns: PostalCode, Borough, and Neighborhood
column_names = ['Postalcode','Borough','Neighborhood']
can_df = pd.DataFrame(columns = column_names)

In [6]:
# Search all the postcode, borough, neighborhood 
for tr_cell in table.find_all('tr'): #Each tr indicates a new row
    row_data=[]
    for td_cell in tr_cell.find_all('td'): #Each td indicates a cell of the row
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        can_df.loc[len(can_df)] = row_data #Append the row to the dataframe

In [7]:
can_df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [8]:
#Remove not assigned Borough
can_df=can_df[can_df['Borough']!='Not assigned']

In [9]:
# Check if there is a 'Not assigned' neighborhood, in that case it will be the same as the borough
if can_df[can_df['Neighborhood']=='Not assigned'].shape[0] != 0:
    can_df[can_df['Neighborhood']=='Not assigned'] = can_df['Borough']

In [10]:
#Add Neighborhoods with a comma when there are two or more with the same Postalcode
merge_df=can_df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
merge_df=merge_df.reset_index(drop=False)
merge_df.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True) #Rename to not have problems merging
merge_df.head()

,Postalcode,Neighborhood_joined
0,M1B,"Malvern, Rouge"
1,M1C,"Rouge Hill, Port Union, Highland Creek"
2,M1E,"Guildwood, Morningside, West Hill"
3,M1G,Woburn
4,M1H,Cedarbrae


In [11]:
df = pd.merge(can_df, merge_df, on='Postalcode') #Merge where Postalcode equals
df.drop(['Neighborhood'],axis=1,inplace=True) #Drop previous Neighborhood column
df.drop_duplicates(inplace=True)
df.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True) #Rename to have the exact asked table
df.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [12]:
df.shape

(103, 3)

In [13]:
#Read the csv file that has the geographical coordinates of each postal code into a dataframe
df_lon_lat = pd.read_csv('http://cocl.us/Geospatial_data')
df_lon_lat.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
df_lon_lat.rename(columns={'Postal Code':'Postalcode'},inplace=True) #Change name to resemble with the other pd
geo_merged = pd.merge(df_lon_lat, df, on='Postalcode') #Merge both dataframes
df_geo=geo_merged[['Postalcode','Borough','Neighborhood','Latitude','Longitude']] #Extract the columns required
df_geo.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
